In [1]:
cd /content/drive/My Drive/Studier/Master/Master Thesis/Coding/

/content/drive/My Drive/Studier/Master/Master Thesis/Coding


In [0]:
# Train
%run ./c-LSTM-LM/train.py

Device type: cuda:0
Loading settings from: ./c-LSTM-LM/settings/settings.txt


------ Parameters -----
   settings_file:  settings.txt
         verbose:  1
            data:  ./dataset_creation/augmented_dataset3_sorted
      checkpoint:  ./c-LSTM-LM/checkpoint_14052020_1300/
          window:  10
       word_size:  20000
        word_dim:  512
      melody_dim:  256
            seed:  0
     num_workers:  4
      num_epochs:  50
      batch_size:  32
              lr:  0.001
    log_interval:  5
      train_rate:  0.8
        val_rate:  0.2
       test_rate:  0
      num_layers:  1


Loading data and creating word and syllable vocabularies.
Currently loading data from: sub_folder1
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder2
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder3
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder4
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder5
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder6
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder7
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder8
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder9
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder10
Number of corrupted files in subfolder: 0
Currently loading data from: sub_folder11
Number of corrupted files in subfolder: 0
Currently loading data from